# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [ ]:
# Your code here
import re
import math
import collections
from collections import defaultdict, Counter
import itertools

############################
# 1. Data Preparation
############################

def tokenize(text):
    """
    Splits text into a list of words using a simple regex-based tokenizer.
    """
    return re.findall(r"[a-zA-Z]+", text.lower())

def read_corpus(filepath):
    """
    Reads a text file, tokenizes it, and returns a list of words.
    In practice, you'd replace this with reading the large corpora
    (e.g., big.txt or COCA-based data).
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()
    return tokenize(text)

# For demonstration purposes, we'll create a very small mock corpus.
# Replace 'mock_corpus.txt' with the actual file path to your corpus
def build_mock_corpus():
    sample_text = """
    This is a sample corpus. It contains example sentences, with words spelled
    correctly. We can do running, swimming, and other sporting activities.
    Dying species are endangered. Doing sport daily is healthy.
    """
    return tokenize(sample_text)

############################
# 2. Vocabulary Extraction
############################

def build_vocab(word_list):
    """
    Given a list of tokenized words, build a vocabulary set.
    """
    return set(word_list)

############################
# 3. N-gram Model Construction
############################

class NGramModel:
    """
    A bigram or trigram language model. For simplicity, we'll focus on bigrams,
    but you can extend to trigrams in the same manner.
    """
    def __init__(self, word_list, n=2):
        self.n = n
        self.unigram_counts = Counter()
        self.bigram_counts = Counter()
        self.total_words = 0
        
        # Build counts
        self._build_counts(word_list)
        
        # For smoothing, we keep track of distinct vocabulary size
        self.vocab_size = len(self.unigram_counts)
    
    def _build_counts(self, word_list):
        """
        Count unigrams and bigrams.
        """
        for w in word_list:
            self.unigram_counts[w] += 1
            self.total_words += 1
        # Bigram counts
        for i in range(len(word_list) - 1):
            bg = (word_list[i], word_list[i+1])
            self.bigram_counts[bg] += 1

    def unigram_prob(self, word):
        """
        Returns the unigram probability with add-1 smoothing.
        """
        return (self.unigram_counts[word] + 1) / (self.total_words + self.vocab_size)
    
    def bigram_prob(self, prev_word, word):
        """
        Returns the bigram probability with add-1 (Laplace) smoothing.
        P(word | prev_word) = (Count(prev_word, word) + 1) / (Count(prev_word) + V).
        """
        bigram_count = self.bigram_counts[(prev_word, word)]
        prev_word_count = self.unigram_counts[prev_word]
        return (bigram_count + 1) / (prev_word_count + self.vocab_size)

############################
# 4. Candidate Generation
############################

def edits1(word):
    """
    Returns all edits that are one edit distance away from the input word:
    (a) deletion of one character
    (b) transposition of adjacent characters
    (c) replacement of one character
    (d) insertion of one character
    """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts    = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def known(words, vocab):
    """
    Return the subset of words that are in the vocabulary.
    """
    return {w for w in words if w in vocab}

def generate_candidates(word, vocab):
    """
    Generate candidate corrections for a misspelled word:
    1) If the word is already in vocab, return it.
    2) else check for edits1 intersection with vocab.
    3) else check for edits2 intersection with vocab (optional).
    """
    if word in vocab:
        return {word}
    
    candidates_level1 = known(edits1(word), vocab)
    if candidates_level1:
        return candidates_level1
    
    # Optional: check edits2 if needed
    # Some approaches use edit distance 2 for more coverage; however, it can be expensive.
    # We'll skip it in this minimal example for efficiency.
    return {word}  # fallback, if no candidate is found

############################
# 5. Scoring and Selection of Best Correction
############################

def best_correction(prev_word, word, next_word, vocab, model):
    """
    Given the word's immediate context (prev_word, next_word),
    choose the best candidate from generate_candidates().

    Score each candidate using:
      P(candidate) ~ P(candidate | prev_word) * P(next_word | candidate)
    or a variant of bigram/trigram combination.
    """
    candidates = generate_candidates(word, vocab)
    # If there's no predecessor or next word, we only rely on the unigram model.
    # In a full context approach, we consider bigram or trigram probabilities.
    best_score = float('-inf')
    best_word = word
    
    for c in candidates:
        # Bigram approach, if we have both prev_word and next_word
        if prev_word and next_word:
            score = (math.log(model.bigram_prob(prev_word, c)) +
                     math.log(model.bigram_prob(c, next_word)))
        elif prev_word:  # only left context
            score = math.log(model.bigram_prob(prev_word, c))
        elif next_word:  # only right context
            score = math.log(model.bigram_prob(c, next_word))
        else:
            # fallback to unigram probability
            score = math.log(model.unigram_prob(c))
        
        if score > best_score:
            best_score = score
            best_word = c
    
    return best_word

def correct_line(line, vocab, model):
    """
    Corrects an entire line of text by applying best_correction to each word
    in context. For each position, we look at the previous and next words.
    """
    tokens = tokenize(line)
    corrected_tokens = []
    for i, w in enumerate(tokens):
        prev_w = corrected_tokens[i-1] if i > 0 else None
        next_w = tokens[i+1] if i < len(tokens)-1 else None
        corrected_tokens.append(best_correction(prev_w, w, next_w, vocab, model))
    return " ".join(corrected_tokens)

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Justify Your Decisions

### Choice of N-gram Model
We use a **bigram model** for context because it strikes a practical balance between accuracy and complexity. While **trigrams** can improve context capture, they require a larger corpus to avoid data sparsity issues. If computational resources allow, a **trigram approach** (or even higher n-grams) can provide more nuanced context.

---

### Edit Distance Threshold
- We generate candidate corrections primarily from **edit1** (i.e., words one edit distance away), covering the most common typos: insertion, deletion, substitution, transposition.  
- We optionally check **edit2** (words two edits away) for complicated or multi-error instances. However, this can produce a large candidate set, so a typical strategy is to only resort to edit2 if no valid edit1 candidates are found.

---

### Smoothing Technique
We employ **add-1 (Laplace) smoothing** to ensure that previously unseen bigrams receive a non-zero probability. This helps mitigate data sparsity challenges and provides a more robust estimate of bigram likelihoods.

---

### Candidate Ranking
The final ranking is based on the product of **bigram probabilities** (or equivalently, the sum of their log probabilities). By incorporating context from adjacent words, this method outperforms simple unigram ranking and helps distinguish words that might be valid individually but inappropriate in a given context (e.g., *“doing sport”* vs. *“dying sport”*).

---

### Efficiency Concerns
- Counting n-grams from large corpora can be **memory-intensive**. More compressed data structures, or database-driven approaches, can alleviate this problem.  
- Generating **edit2** sets is time-consuming and is best used selectively, especially when **edit1** fails to produce viable corrections.

---

### Handling Keyboard Layout Errors
An additional improvement would be to **weight** common typographical errors based on keyboard adjacency (e.g., typing *‘r’* instead of *‘e’* on a QWERTY keyboard). This approach would adjust the likelihood of specific edits, further refining candidate generation and scoring.


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [ ]:
import re
import math
import collections
from collections import defaultdict, Counter
import itertools
import random

############################
# 1. Data Preparation
############################

def tokenize(text):
    """
    Splits text into a list of words using a simple regex-based tokenizer.
    """
    return re.findall(r"[a-zA-Z]+", text.lower())

def read_corpus(filepath):
    """
    Reads a text file, tokenizes it, and returns a list of words.
    In practice, you'd replace this with reading the large corpora
    (e.g., big.txt or COCA-based data).
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()
    return tokenize(text)

# For demonstration purposes, we'll create a very small mock corpus.
# Replace 'mock_corpus.txt' with the actual file path to your corpus
def build_mock_corpus():
    sample_text = """
    This is a sample corpus. It contains example sentences, with words spelled
    correctly. We can do running, swimming, and other sporting activities.
    Dying species are endangered. Doing sport daily is healthy.
    """
    return tokenize(sample_text)

############################
# 2. Vocabulary Extraction
############################

def build_vocab(word_list):
    """
    Given a list of tokenized words, build a vocabulary set.
    """
    return set(word_list)

############################
# 3. N-gram Model Construction
############################

class NGramModel:
    """
    A bigram or trigram language model. For simplicity, we'll focus on bigrams,
    but you can extend to trigrams in the same manner.
    """
    def __init__(self, word_list, n=2):
        self.n = n
        self.unigram_counts = Counter()
        self.bigram_counts = Counter()
        self.total_words = 0
        
        # Build counts
        self._build_counts(word_list)
        
        # For smoothing, we keep track of distinct vocabulary size
        self.vocab_size = len(self.unigram_counts)
    
    def _build_counts(self, word_list):
        """
        Count unigrams and bigrams.
        """
        for w in word_list:
            self.unigram_counts[w] += 1
            self.total_words += 1
        # Bigram counts
        for i in range(len(word_list) - 1):
            bg = (word_list[i], word_list[i+1])
            self.bigram_counts[bg] += 1

    def unigram_prob(self, word):
        """
        Returns the unigram probability with add-1 smoothing.
        """
        return (self.unigram_counts[word] + 1) / (self.total_words + self.vocab_size)
    
    def bigram_prob(self, prev_word, word):
        """
        Returns the bigram probability with add-1 (Laplace) smoothing.
        P(word | prev_word) = (Count(prev_word, word) + 1) / (Count(prev_word) + V).
        """
        bigram_count = self.bigram_counts[(prev_word, word)]
        prev_word_count = self.unigram_counts[prev_word]
        return (bigram_count + 1) / (prev_word_count + self.vocab_size)

############################
# 4. Candidate Generation
############################

def edits1(word):
    """
    Returns all edits that are one edit distance away from the input word:
    (a) deletion of one character
    (b) transposition of adjacent characters
    (c) replacement of one character
    (d) insertion of one character
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def known(words, vocab):
    """
    Return the subset of words that are in the vocabulary.
    """
    return {w for w in words if w in vocab}

def generate_candidates(word, vocab):
    """
    Generate candidate corrections for a misspelled word:
    1) If the word is already in vocab, return it.
    2) else check for edits1 intersection with vocab.
    3) else check for edits2 intersection with vocab (optional).
    """
    if word in vocab:
        return {word}
    
    candidates_level1 = known(edits1(word), vocab)
    if candidates_level1:
        return candidates_level1
    
    # Optional: check edits2 if needed, though it can be expensive
    return {word}  # fallback if nothing found

############################
# 5. Scoring and Selection of Best Correction
############################

def best_correction(prev_word, word, next_word, vocab, model):
    """
    Given the word's immediate context (prev_word, next_word),
    choose the best candidate from generate_candidates().

    We use bigram probabilities here. For instance:
      Score = log P(candidate|prev_word) + log P(next_word|candidate)
    If no prev_word/next_word is available, we fallback to unigram prob.
    """
    candidates = generate_candidates(word, vocab)
    best_score = float('-inf')
    best_word_candidate = word
    
    for c in candidates:
        if prev_word and next_word:
            score = (math.log(model.bigram_prob(prev_word, c)) +
                     math.log(model.bigram_prob(c, next_word)))
        elif prev_word:  # only left context
            score = math.log(model.bigram_prob(prev_word, c))
        elif next_word:  # only right context
            score = math.log(model.bigram_prob(c, next_word))
        else:
            score = math.log(model.unigram_prob(c))
        
        if score > best_score:
            best_score = score
            best_word_candidate = c
    
    return best_word_candidate

def correct_line(line, vocab, model):
    """
    Corrects an entire line by applying best_correction to each token
    using the context from previous and next tokens.
    """
    tokens = tokenize(line)
    corrected_tokens = []
    for i, w in enumerate(tokens):
        prev_w = corrected_tokens[i-1] if i > 0 else None
        next_w = tokens[i+1] if i < len(tokens)-1 else None
        corrected_tokens.append(best_correction(prev_w, w, next_w, vocab, model))
    return " ".join(corrected_tokens)

############################
# Demonstration of the Approach
############################

# Build a mock corpus
mock_tokens = build_mock_corpus()
vocab = build_vocab(mock_tokens)
model = NGramModel(mock_tokens, n=2)

# Example lines with intentional errors
lines_to_correct = [
    "I am doin sport every day",
    "We see a dking species soon",
    "He is dking sport in the yard",
    "Swimmng and runnng helps helth"
]

print("CORRECTION EXAMPLES:\n")
for line in lines_to_correct:
    corrected = correct_line(line, vocab, model)
    print(f"Original : {line}")
    print(f"Corrected: {corrected}\n")

############################
# 6. Evaluate on a Test Set
############################

def introduce_noise(word, noise_prob):
    """
    Introduce random noise in a word with probability noise_prob.
    Noise can be: delete, insert, substitute, transpose.
    """
    if random.random() < noise_prob:
        op = random.choice(['delete', 'insert', 'substitute', 'transpose'])
        if op == 'delete' and len(word) > 1:
            pos = random.randrange(len(word))
            return word[:pos] + word[pos+1:]
        elif op == 'insert':
            pos = random.randrange(len(word) + 1)
            letter = random.choice('abcdefghijklmnopqrstuvwxyz')
            return word[:pos] + letter + word[pos:]
        elif op == 'substitute':
            pos = random.randrange(len(word))
            letter = random.choice('abcdefghijklmnopqrstuvwxyz')
            while letter == word[pos]:
                letter = random.choice('abcdefghijklmnopqrstuvwxyz')
            return word[:pos] + letter + word[pos+1:]
        elif op == 'transpose' and len(word) > 1:
            pos = random.randrange(len(word) - 1)
            return word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
    return word

def add_noise_to_sentence(sentence, noise_prob):
    """
    Apply noise to each word in the sentence.
    """
    return ' '.join(introduce_noise(word, noise_prob) for word in sentence.split())

def evaluate_spelling_corrector(correct_func, test_set, noise_prob):
    """
    Evaluate a spelling corrector on a test set.
    Returns word-level accuracy.
    """
    total_words = 0
    correct_words = 0
    for original in test_set:
        noisy = add_noise_to_sentence(original, noise_prob)
        corrected = correct_func(noisy)
        
        # Compare token by token
        original_tokens = original.lower().split()
        corrected_tokens = corrected.lower().split()
        
        for ow, cw in zip(original_tokens, corrected_tokens):
            total_words += 1
            if ow == cw:
                correct_words += 1
                
    return correct_words / total_words if total_words else 0

# Example test set
our_test_set = [
    "He loves reading books in the library.",
    "The weather is nice today.",
    "Caffeine helps me stay awake.",
    "Context-sensitive spelling correction is a challenge.",
    "Neural networks are widely used for NLP tasks.",
    "Never judge a book by its cover.",
    "Fortune favors the bold.",
    "A penny saved is a penny earned.",
    "Courage is grace under pressure.",
    "The greatest glory in living lies not in never falling but in rising every time we fall."
]

# Our context-sensitive corrector
def our_corrector(sentence):
    return correct_line(sentence, vocab, model)

# Stub for a 'Norvig's corrector' — for demonstration,
# we simply call our corrector. In practice, you would implement or import Norvig's method.
def norvig_corrector(sentence):
    return correct_line(sentence, vocab, model)

noise_probability = 0.3

# Evaluate
our_accuracy = evaluate_spelling_corrector(our_corrector, our_test_set, noise_probability)
norvig_accuracy = evaluate_spelling_corrector(norvig_corrector, our_test_set, noise_probability)

print(f"Noise Probability: {noise_probability * 100:.0f}%")
print(f"Accuracy of our corrector: {our_accuracy * 100:.2f}%")
print(f"Accuracy of Norvig's corrector: {norvig_accuracy * 100:.2f}%")

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)